<a href="https://colab.research.google.com/github/sahithim1/CoronaCases/blob/master/corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install plotly


import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
from datetime import timedelta
import math
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib as mpl
import seaborn as sns
mpl.rcParams['figure.dpi']= 600
sns.set() 
df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [3]:
def clean_cols(df, rates=False, smooth_days=3):
    try:
        df['Date'] = pd.to_datetime(df.Date).dt.date
        df['EpidemicStartDate'] = df.sort_values(by='Date').loc[df.Confirmed!=0].Date.iloc[0]
        df['DaysElapsed'] = (df.Date - df.EpidemicStartDate).dt.days + 1
    except:
        print('Unable to convert dates properly!')
    df['NewConfirmed'] = df.Confirmed.diff(periods=1)
    # Takes the natural log not log10 s
    df['DaysElapsed_Log'] = np.log(df.DaysElapsed)
    df['Confirmed_Log'] = np.log(df.Confirmed)
    df['NewConfirmed_Log'] = np.log(df.NewConfirmed)
    if rates:
        # Haha idk math
        df['GrowthRate'] = df.NewConfirmed/df.NewConfirmed.shift(1)
        df['Ratio'] = df.Confirmed/df.Confirmed.shift(1)
        # Haha irdk math
        df['GrowthRate_Smooth'] = df.GrowthRate.rolling(smooth_days).sum()/smooth_days
        df['Ratio_smooth'] = df.Ratio.rolling(smooth_days.sum())/smooth_days
    return df

temp = df.drop(['Lat','Long'],axis=1).groupby(by='Country/Region').sum().reset_index()
countries = pd.melt(temp, id_vars=['Country/Region'], var_name='date', value_name='cases')
countries.columns = ['Country','Date','Confirmed']
cleaned_countries = countries.groupby('Country').apply(clean_cols)



/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

invalid value encountered in log



In [4]:
cleaned_countries.head()

,Country,Date,Confirmed,EpidemicStartDate,DaysElapsed,NewConfirmed,DaysElapsed_Log,Confirmed_Log,NewConfirmed_Log
0,Afghanistan,2020-01-22,0,2020-02-24,-32,NaN,NaN,-inf,NaN
1,Albania,2020-01-22,0,2020-03-09,-46,NaN,NaN,-inf,NaN
2,Algeria,2020-01-22,0,2020-02-25,-33,NaN,NaN,-inf,NaN
3,Andorra,2020-01-22,0,2020-03-02,-39,NaN,NaN,-inf,NaN
4,Angola,2020-01-22,0,2020-03-20,-57,NaN,NaN,-inf,NaN


In [5]:
cleaned_countries[cleaned_countries["Country"] == "US"]

,Country,Date,Confirmed,EpidemicStartDate,DaysElapsed,NewConfirmed,DaysElapsed_Log,Confirmed_Log,NewConfirmed_Log
174,US,2020-01-22,1,2020-01-22,1,NaN,0.000000,0.000000,NaN
362,US,2020-01-23,1,2020-01-22,2,0.0,0.693147,0.000000,-inf
550,US,2020-01-24,2,2020-01-22,3,1.0,1.098612,0.693147,0.000000
738,US,2020-01-25,2,2020-01-22,4,0.0,1.386294,0.693147,-inf
926,US,2020-01-26,5,2020-01-22,5,3.0,1.609438,1.609438,1.098612
...,...,...,...,...,...,...,...,...,...
39090,US,2020-08-16,5403213,2020-01-22,208,42048.0,5.337538,15.502504,10.646567
39278,US,2020-08-17,5438325,2020-01-22,209,35112.0,5.342334,15.508982,10.466298
39466,US,2020-08-18,5482416,2020-01-22,210,44091.0,5.347108,15.517056,10.694011
39654,US,2020-08-19,5529824,2020-01-22,211,47408.0,5.351858,15.525667,10.766546


In [7]:
px.scatter(data_frame=cleaned_countries[cleaned_countries.Country.isin(['Canada','US','Mexico'])],
           x='Date', y='NewConfirmed', color='Country', symbol='Country',
           log_x=False,log_y=True, 
           
           title='New Confirmed COVID-19 Cases in North America', template='ggplot2', 
          )

In [ ]:
ncleaned_countries[cleaned_countries.Country.isin(countries_list)].tail()
